In [ ]:
repo_url = 'https://github.com/fraimwork/fraimwork'

In [ ]:
from utils.gitutils import clone_repo

# Clone the repo
repo = clone_repo(repo_url, max_size_mb=10000)
local_repo_path = str(repo.working_dir)

In [ ]:
from utils.filetreeutils import FileTree

(file_tree := FileTree.from_dir(f'{local_repo_path}'))

In [ ]:
from utils.agent import Agent, GenerationConfig
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv('API_KEY')

classifier = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="classifier",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a software engineer tasked with classifying the framework of a codebase. \
In the following prompts, you will be given the subdirectories of a codebase. You can respond with one of the following:
- IDENTIFY: FRAMEWORK_NAME (Only select this if you are absolutely certain)
- N/A (if you cannot identify the framework based on the subdirectories)"""
)

In [ ]:
import re
from utils.agent import Interaction
from utils.languageutils import get_imports
# RegEx to match classifier responses
identify_re = re.compile(r"IDENTIFY: (.+)")
search_re = re.compile(r"OPEN: (.+)")
unable_re = re.compile(r"N/A")

def classify_node(node):
    if node not in file_tree.nodes:
        print(f"Node {node} not found in file tree")
        return
    node_dict = file_tree.nodes[node]
    neighbors = list(file_tree[node])
    prompt = f"{file_tree.nodes[node]['name']}\n" + '\n'.join(f"├── {file_tree.nodes[k]['name']}" for k in neighbors)
    all_imports = [f'Dependencies of {neighbor}\n----\n' + '\n'.join(imports) for neighbor in neighbors if 'content' in file_tree.nodes[neighbor] and (imports := get_imports(file_tree.nodes[neighbor]['path']))]
    context = [Interaction('...', '\n'.join(imports for imports in all_imports))]
    response = classifier.chat(prompt, custom_context=context)
    # Match the response to the RegEx
    if (m := identify_re.match(response)):
        print(f"{node_dict['name']} is a {m.group(1)} project")
        return m.group(1).strip()
    elif (m := search_re.match(response)):
        print(f"Opening {m.group(1)} for {node_dict['name']} project")
        to_search = f'{node}\\{m.group(1).strip()}'
        if node == file_tree.root_node(): to_search = to_search[len(file_tree.root_node()):]
        return classify_node(to_search)
    elif (m := unable_re.match(response)):
        print(f"Searching {node} ...")
        return [classify_node(node) for node in neighbors if 'content' not in file_tree.nodes[node]]
    else:
        print(f"Classifier response: {response}")

(root_classification := classify_node(file_tree.root_node()))